In [1]:
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.pipeline import Pipeline

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append("..") 
from gcforest.gcforest import GCForest

import xlwt
import itertools

# 数据及参数

In [2]:
random_seed = 42
cv=5
score = 'f1_weighted'

In [3]:
def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = random_seed
    ca_config["max_layers"] = 10
    ca_config["early_stopping_rounds"] = 3
    ca_config["n_classes"] = 6
    ca_config["estimators"] = []
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"})
    config["cascade"] = ca_config
    return config

In [4]:
path = os.getcwd()+'/../data/20122018freshwater_four_feature.csv'
data = pd.read_csv(path, na_values = np.nan)

In [5]:
X = data.drop(['本周水质'], axis=1).values # Series
y = data['本周水质'].values.reshape(-1,1) - 1

# 1. 中位数填充缺失值，2.Z-score标准化
clean_pipeline = Pipeline([('imputer', preprocessing.Imputer(missing_values='NaN',strategy="median")),
                           ('std_scaler', preprocessing.StandardScaler())])
X = clean_pipeline.fit_transform(X)

In [6]:
X.shape

(33612, 4)

In [7]:
comb_3=list(itertools.combinations([0,1,2,3],3))
comb_2=list(itertools.combinations([0,1,2,3],2))

# k折交叉验证

In [8]:
# function：使用5折交叉验证统计各类别5次平均后的Acc，5次平均后的F1，和模型的总Acc以及总Weighted F1

# Input: 
#     X: 总样本
#     y: 总样本
#     model: function
#     cv: cross_validation的次数
# Output:
#     Acc_mean, 各类别的Acc
#     F1_mean, 各类别F1
#     Support_mean, 各类别预测样本占总样本的比重
#     Acc, 总Acc
#     F1_weighted 总Weighted F1
# function：使用5折交叉验证统计各类别5次平均后的Acc，5次平均后的F1，和模型的总Acc以及总Weighted F1

# Input: 
#     X: 总样本
#     y: 总样本
#     model: function
#     cv: cross_validation的次数
# Output:
#     Acc_mean, 各类别的Acc
#     F1_mean, 各类别F1
#     Support_mean, 各类别预测样本占总样本的比重
#     Acc, 总Acc
#     F1_weighted 总Weighted F1


def kftrain(X, y, model, cv):
    model_name = model.__class__.__name__

    n_samples = X.shape[0]
    n_features = X.shape[1]
    n_class = np.unique(y).shape[0]

    _Acc_matrix = np.zeros((n_class, cv))
    _Pr_matrix = np.zeros((n_class, cv))
    _Rc_matrix = np.zeros((n_class, cv))
    _F1_matrix = np.zeros((n_class, cv))
    _Support_matrix = np.zeros((n_class, cv))

    Acc_matrix = np.zeros((n_class, cv))
    Pr_matrix = np.zeros((n_class, cv))
    Rc_matrix = np.zeros((n_class, cv))
    F1_matrix = np.zeros((n_class, cv))
    Support_matrix = np.zeros((n_class, cv))
    
    k = 0
    skf = StratifiedKFold(n_splits=cv) # 定义5折分层划分器
    
    for train_index, test_index in skf.split(X, y):
        K_train_x, K_test_x = X[train_index], X[test_index]
        K_train_y, K_test_y = y[train_index], y[test_index]
        if model_name == 'GCForest':
            model.fit_transform(K_train_x, K_train_y.reshape(K_train_y.shape[0]))
        else:
            model.fit(K_train_x, K_train_y)
    
        K_test_y_pred = model.predict(K_test_x)
        K_train_y_pred = model.predict(K_train_x)
        
        # 由混淆矩阵计算各类别的Acc
        test_cm = confusion_matrix(K_test_y, K_test_y_pred)
        train_cm = confusion_matrix(K_train_y, K_train_y_pred)
        test_acc_all_class = np.zeros(n_class)
        train_acc_all_class = np.zeros(n_class)
        
        i = 0
        for c in test_cm:
            test_acc_all_class[i] = c[i]/np.sum(c)
            i += 1
        Acc_matrix[:,k] = test_acc_all_class
        
        i = 0
        for c in train_cm:
            train_acc_all_class[i] = c[i]/np.sum(c)
            i += 1
        _Acc_matrix[:,k] = train_acc_all_class
        
        # 由classification_report提取各类别的precision, recall, F1
        cr = classification_report(K_test_y, K_test_y_pred, digits=4)
        _cr = classification_report(K_train_y, K_train_y_pred, digits=4)
        pr_all_class = np.zeros(n_class)
        rc_all_class = np.zeros(n_class)
        f1_all_class = np.zeros(n_class)
        _pr_all_class = np.zeros(n_class)
        _rc_all_class = np.zeros(n_class)
        _f1_all_class = np.zeros(n_class)
        
        support_all_class = np.zeros(n_class)
        _support_all_class = np.zeros(n_class)
        
        i = 0
        for l in range(2,8):
            pr_all_class[i] = float(cr.splitlines()[l].split()[1])
            _pr_all_class[i] = float(_cr.splitlines()[l].split()[1])
            rc_all_class[i] = float(cr.splitlines()[l].split()[2])
            _rc_all_class[i] = float(_cr.splitlines()[l].split()[2])
            f1_all_class[i] = float(cr.splitlines()[l].split()[3])
            _f1_all_class[i] = float(_cr.splitlines()[l].split()[3])
            
            support_all_class[i] = float(cr.splitlines()[l].split()[4])/(n_samples/cv)
            _support_all_class[i] = float(_cr.splitlines()[l].split()[4])/(n_samples*(1-1/cv))
            i = i + 1
        Pr_matrix[:,k] = pr_all_class
        Rc_matrix[:,k] = rc_all_class
        F1_matrix[:,k] = f1_all_class
        Support_matrix[:,k] = support_all_class

        _Pr_matrix[:,k] = _pr_all_class
        _Rc_matrix[:,k] = _rc_all_class
        _F1_matrix[:,k] = _f1_all_class
        _Support_matrix[:,k] = _support_all_class
        
        k += 1
    
    return Acc_matrix, Pr_matrix, Rc_matrix, F1_matrix, Support_matrix, _Acc_matrix, _Pr_matrix, _Rc_matrix, _F1_matrix, _Support_matrix

In [9]:
config = get_toy_config()

model = GCForest(config)

for cols in comb_3:
    print("DCF"+str(cols))
    X_new = X[:,cols]
    workbook = xlwt.Workbook(encoding = 'utf-8')  

    model_name = model.__class__.__name__
    print(model_name)
    Acc_matrix, Pr_matrix, Rc_matrix, F1_matrix, Support_matrix, _Acc_matrix, _Pr_matrix, _Rc_matrix, _F1_matrix, _Support_matrix = kftrain(X_new, y, model, cv)

    Acc_cv = np.mean(Acc_matrix, axis=0)
    Pr_cv = np.mean(Pr_matrix, axis=0)
    Rc_cv = np.mean(Rc_matrix, axis=0)
    F1_weighted_cv = np.sum(F1_matrix*Support_matrix, axis=0)
    _Acc_cv = np.mean(_Acc_matrix, axis=0)
    _Pr_cv = np.mean(_Pr_matrix, axis=0)
    _Rc_cv = np.mean(_Rc_matrix, axis=0)
    _F1_weighted_cv = np.sum(_F1_matrix*_Support_matrix, axis=0)

    Acc_mean = np.mean(Acc_matrix, axis=1)
    Pr_mean = np.mean(Pr_matrix, axis=1)
    Rc_mean = np.mean(Rc_matrix, axis=1)
    F1_mean = np.mean(F1_matrix, axis=1)
    Acc_SD = np.std(Acc_matrix, axis=1)
    Pr_SD = np.std(Pr_matrix, axis=1)
    Rc_SD = np.std(Rc_matrix, axis=1)
    F1_SD = np.std(F1_matrix, axis=1)

    _Acc_mean = np.mean(_Acc_matrix, axis=1)
    _Pr_mean = np.mean(_Pr_matrix, axis=1)
    _Rc_mean = np.mean(_Rc_matrix, axis=1)
    _F1_mean = np.mean(_F1_matrix, axis=1)
    _Acc_SD = np.std(_Acc_matrix, axis=1)
    _Pr_SD = np.std(_Pr_matrix, axis=1)
    _Rc_SD = np.std(_Rc_matrix, axis=1)
    _F1_SD = np.std(_F1_matrix, axis=1)
    
    worksheet = workbook.add_sheet(model_name, cell_overwrite_ok=True)
    style = xlwt.XFStyle()
    pattern = xlwt.Pattern()
    pattern.pattern_fore_colour = xlwt.Style.colour_map['yellow']
    style.pattern = pattern
    
    l=0
    worksheet.write(l,0, "train_Acc_matrix")
    worksheet.write(l,cv+2, "test_Acc_matrix")

    for i in range(6):
        l+=1
        for j in range(cv):
            worksheet.write(l, j, _Acc_matrix[i,j])
            worksheet.write(l, j+cv+2, Acc_matrix[i,j])
        worksheet.write(l, j+1, _Acc_mean[i]) 
        worksheet.write(l, j+2, _Acc_SD[i])
        worksheet.write(l, j+2+cv+1, Acc_mean[i])
        worksheet.write(l, j+2+cv+2, Acc_SD[i])
    l+=1
    for j in range(cv):
        worksheet.write(l, j, _Acc_cv[j])
        worksheet.write(l, j+cv+2, Acc_cv[j])
    worksheet.write(l, j+1, np.mean(_Acc_cv), style=style)
    worksheet.write(l, j+2, np.std(_Acc_cv), style=style)
    worksheet.write(l, j+2+cv+1, np.mean(Acc_cv), style=style)
    worksheet.write(l, j+2+cv+2, np.std(Acc_cv), style=style)
    #=======#
    l+=3
    worksheet.write(l,0, "train_Pr_matrix")
    worksheet.write(l,cv+2, "test_Pr_matrix")

    for i in range(6):
        l+=1
        for j in range(cv):
            worksheet.write(l, j, _Pr_matrix[i,j])
            worksheet.write(l, j+cv+2, Pr_matrix[i,j])
        worksheet.write(l, j+1, _Pr_mean[i]) 
        worksheet.write(l, j+2, _Pr_SD[i])
        worksheet.write(l, j+2+cv+1, Pr_mean[i])
        worksheet.write(l, j+2+cv+2, Pr_SD[i])
    l+=1
    for j in range(cv):
        worksheet.write(l, j, _Pr_cv[j])
        worksheet.write(l, j+cv+2, Pr_cv[j])
    worksheet.write(l, j+1, np.mean(_Pr_cv), style=style)
    worksheet.write(l, j+2, np.std(_Pr_cv), style=style)
    worksheet.write(l, j+2+cv+1, np.mean(Pr_cv), style=style)
    worksheet.write(l, j+2+cv+2, np.std(Pr_cv), style=style)

    #=======#
    l+=3
    worksheet.write(l,0, "train_Rc_matrix")
    worksheet.write(l,cv+2, "test_Rc_matrix")

    for i in range(6):
        l+=1
        for j in range(cv):
            worksheet.write(l, j, _Rc_matrix[i,j])
            worksheet.write(l, j+cv+2, Rc_matrix[i,j])
        worksheet.write(l, j+1, _Rc_mean[i]) 
        worksheet.write(l, j+2, _Rc_SD[i])
        worksheet.write(l, j+2+cv+1, Rc_mean[i])
        worksheet.write(l, j+2+cv+2, Rc_SD[i])
    l+=1
    for j in range(cv):
        worksheet.write(l, j, _Rc_cv[j])
        worksheet.write(l, j+cv+2, Rc_cv[j])
    worksheet.write(l, j+1, np.mean(_Rc_cv), style=style)
    worksheet.write(l, j+2, np.std(_Rc_cv), style=style)
    worksheet.write(l, j+2+cv+1, np.mean(Rc_cv), style=style)
    worksheet.write(l, j+2+cv+2, np.std(Rc_cv), style=style)
    #=======#
    l+=3
    worksheet.write(l,0, "train_F1_matrix")
    worksheet.write(l,cv+2, "test_F1_matrix")

    for i in range(6):
        l+=1
        for j in range(cv):
            worksheet.write(l, j, _F1_matrix[i,j])
            worksheet.write(l, j+cv+2, F1_matrix[i,j])
        worksheet.write(l, j+1, _F1_mean[i]) 
        worksheet.write(l, j+2, _F1_SD[i])
        worksheet.write(l, j+2+cv+1, F1_mean[i])
        worksheet.write(l, j+2+cv+2, F1_SD[i])
    l+=1
    for j in range(cv):
        worksheet.write(l, j, _F1_weighted_cv[j])
        worksheet.write(l, j+cv+2, F1_weighted_cv[j])
    worksheet.write(l, j+1, np.mean(_F1_weighted_cv), style=style)
    worksheet.write(l, j+2, np.std(_F1_weighted_cv), style=style)
    worksheet.write(l, j+2+cv+1, np.mean(F1_weighted_cv), style=style)
    worksheet.write(l, j+2+cv+2, np.std(F1_weighted_cv), style=style)


    workbook.save("../res/DCF"+str(cols)+".xls")

[ 2019-07-05 23:31:18,390][cascade_classifier.fit_transform] X_groups_train.shape=[(26886, 3)],y_train.shape=(26886,),X_groups_test.shape=no_test,y_test.shape=no_test
[ 2019-07-05 23:31:18,391][cascade_classifier.fit_transform] group_dims=[3]
[ 2019-07-05 23:31:18,392][cascade_classifier.fit_transform] group_starts=[0]
[ 2019-07-05 23:31:18,393][cascade_classifier.fit_transform] group_ends=[3]
[ 2019-07-05 23:31:18,395][cascade_classifier.fit_transform] X_train.shape=(26886, 3),X_test.shape=(0, 3)
[ 2019-07-05 23:31:18,397][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(26886, 3), X_cur_test.shape=(0, 3)


DCF(0, 1, 2)
GCForest


/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
[ 2019-07-05 23:31:18,681][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_0.predict)=83.36%
[ 2019-07-05 23:31:18,908][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_1.predict)=83.29%
[ 2019-07-05 23:31:19,110][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_2.predict)=83.14%
[ 2019-07-05 23:31:19,298][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_3.predict)=83.06%
[ 2019-07-05 23:31:19,480][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_4.predict)=83.21%
[ 2019-07-05 23:31:19,487][kfold_wrapper.log_eval_metrics] Weighted F1 (l

[ 2019-07-05 23:31:28,784][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_1 - 5_folds.train_4.predict)=84.08%
[ 2019-07-05 23:31:28,791][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_1 - 5_folds.train_cv.predict)=84.01%
[ 2019-07-05 23:31:29,262][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_0.predict)=83.98%
[ 2019-07-05 23:31:29,727][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_1.predict)=83.34%
[ 2019-07-05 23:31:30,168][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_2.predict)=83.79%
[ 2019-07-05 23:31:30,630][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_3.predict)=83.92%
[ 2019-07-05 23:31:31,081][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_4.predict)=83.72%
[ 2019-07-05 23:31:31,089][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds

[ 2019-07-05 23:31:56,219][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_0.predict)=77.11%
[ 2019-07-05 23:31:56,698][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_1.predict)=76.01%
[ 2019-07-05 23:31:57,157][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_2.predict)=76.00%
[ 2019-07-05 23:31:57,605][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_3.predict)=75.24%
[ 2019-07-05 23:31:58,072][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_4.predict)=75.19%
[ 2019-07-05 23:31:58,079][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_cv.predict)=75.91%
[ 2019-07-05 23:31:58,572][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_0.predict)=75.66%
[ 2019-07-05 23:31:59,009][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds

...

[ 2019-07-06 00:08:41,648][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_0 - 5_folds.train_2.predict)=98.11%
[ 2019-07-06 00:08:41,984][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_0 - 5_folds.train_3.predict)=98.53%
[ 2019-07-06 00:08:42,435][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_0 - 5_folds.train_4.predict)=98.35%
[ 2019-07-06 00:08:42,449][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_0 - 5_folds.train_cv.predict)=98.35%
[ 2019-07-06 00:08:42,818][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_1 - 5_folds.train_0.predict)=98.52%
[ 2019-07-06 00:08:43,169][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_1 - 5_folds.train_1.predict)=98.34%
[ 2019-07-06 00:08:43,558][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_1 - 5_folds.train_2.predict)=98.38%
[ 2019-07-06 00:08:43,866][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_1 - 5_folds

[ 2019-07-06 00:09:01,579][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_2 - 5_folds.train_4.predict)=98.24%
[ 2019-07-06 00:09:01,585][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_2 - 5_folds.train_cv.predict)=98.34%
[ 2019-07-06 00:09:01,880][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_3 - 5_folds.train_0.predict)=98.46%
[ 2019-07-06 00:09:02,160][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_3 - 5_folds.train_1.predict)=98.58%
[ 2019-07-06 00:09:02,429][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_3 - 5_folds.train_2.predict)=98.27%
[ 2019-07-06 00:09:02,735][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_3 - 5_folds.train_3.predict)=98.13%
[ 2019-07-06 00:09:03,026][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_3 - 5_folds.train_4.predict)=98.50%
[ 2019-07-06 00:09:03,033][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_3 - 5_folds

[ 2019-07-06 00:09:19,136][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_5 - 5_folds.train_0.predict)=96.82%
[ 2019-07-06 00:09:19,436][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_5 - 5_folds.train_1.predict)=96.85%
[ 2019-07-06 00:09:19,803][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_5 - 5_folds.train_2.predict)=96.60%
[ 2019-07-06 00:09:20,257][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_5 - 5_folds.train_3.predict)=96.92%
[ 2019-07-06 00:09:20,680][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_5 - 5_folds.train_4.predict)=97.14%
[ 2019-07-06 00:09:20,689][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_5 - 5_folds.train_cv.predict)=96.87%
[ 2019-07-06 00:09:21,026][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_6 - 5_folds.train_0.predict)=97.08%
[ 2019-07-06 00:09:21,319][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_6 - 5_folds

[ 2019-07-06 00:09:39,009][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_7 - 5_folds.train_2.predict)=98.51%
[ 2019-07-06 00:09:39,737][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_7 - 5_folds.train_3.predict)=98.48%
[ 2019-07-06 00:09:40,467][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_7 - 5_folds.train_4.predict)=98.34%
[ 2019-07-06 00:09:40,473][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_7 - 5_folds.train_cv.predict)=98.36%
[ 2019-07-06 00:09:40,482][cascade_classifier.calc_f1] Weighted F1 (layer_4 - train.classifier_average)=98.38%
[ 2019-07-06 00:09:40,488][cascade_classifier.fit_transform] [layer=5] look_indexs=[0], X_cur_train.shape=(26892, 51), X_cur_test.shape=(0, 51)
[ 2019-07-06 00:09:40,798][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_0 - 5_folds.train_0.predict)=98.40%
[ 2019-07-06 00:09:41,065][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_0 - 5_folds.

[ 2019-07-06 00:09:56,353][cascade_classifier.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(26892, 51)


In [10]:
model = GCForest(config)

for cols in comb_2:
    print("DCF"+str(cols))
    X_new = X[:,cols]
    workbook = xlwt.Workbook(encoding = 'utf-8')  

    model_name = model.__class__.__name__
    print(model_name)
    Acc_matrix, Pr_matrix, Rc_matrix, F1_matrix, Support_matrix, _Acc_matrix, _Pr_matrix, _Rc_matrix, _F1_matrix, _Support_matrix = kftrain(X_new, y, model, cv)

    Acc_cv = np.mean(Acc_matrix, axis=0)
    Pr_cv = np.mean(Pr_matrix, axis=0)
    Rc_cv = np.mean(Rc_matrix, axis=0)
    F1_weighted_cv = np.sum(F1_matrix*Support_matrix, axis=0)
    _Acc_cv = np.mean(_Acc_matrix, axis=0)
    _Pr_cv = np.mean(_Pr_matrix, axis=0)
    _Rc_cv = np.mean(_Rc_matrix, axis=0)
    _F1_weighted_cv = np.sum(_F1_matrix*_Support_matrix, axis=0)

    Acc_mean = np.mean(Acc_matrix, axis=1)
    Pr_mean = np.mean(Pr_matrix, axis=1)
    Rc_mean = np.mean(Rc_matrix, axis=1)
    F1_mean = np.mean(F1_matrix, axis=1)
    Acc_SD = np.std(Acc_matrix, axis=1)
    Pr_SD = np.std(Pr_matrix, axis=1)
    Rc_SD = np.std(Rc_matrix, axis=1)
    F1_SD = np.std(F1_matrix, axis=1)

    _Acc_mean = np.mean(_Acc_matrix, axis=1)
    _Pr_mean = np.mean(_Pr_matrix, axis=1)
    _Rc_mean = np.mean(_Rc_matrix, axis=1)
    _F1_mean = np.mean(_F1_matrix, axis=1)
    _Acc_SD = np.std(_Acc_matrix, axis=1)
    _Pr_SD = np.std(_Pr_matrix, axis=1)
    _Rc_SD = np.std(_Rc_matrix, axis=1)
    _F1_SD = np.std(_F1_matrix, axis=1)
    
    worksheet = workbook.add_sheet(model_name, cell_overwrite_ok=True)
    style = xlwt.XFStyle()
    pattern = xlwt.Pattern()
    pattern.pattern_fore_colour = xlwt.Style.colour_map['yellow']
    style.pattern = pattern
    
    l=0
    worksheet.write(l,0, "train_Acc_matrix")
    worksheet.write(l,cv+2, "test_Acc_matrix")

    for i in range(6):
        l+=1
        for j in range(cv):
            worksheet.write(l, j, _Acc_matrix[i,j])
            worksheet.write(l, j+cv+2, Acc_matrix[i,j])
        worksheet.write(l, j+1, _Acc_mean[i]) 
        worksheet.write(l, j+2, _Acc_SD[i])
        worksheet.write(l, j+2+cv+1, Acc_mean[i])
        worksheet.write(l, j+2+cv+2, Acc_SD[i])
    l+=1
    for j in range(cv):
        worksheet.write(l, j, _Acc_cv[j])
        worksheet.write(l, j+cv+2, Acc_cv[j])
    worksheet.write(l, j+1, np.mean(_Acc_cv), style=style)
    worksheet.write(l, j+2, np.std(_Acc_cv), style=style)
    worksheet.write(l, j+2+cv+1, np.mean(Acc_cv), style=style)
    worksheet.write(l, j+2+cv+2, np.std(Acc_cv), style=style)
    #=======#
    l+=3
    worksheet.write(l,0, "train_Pr_matrix")
    worksheet.write(l,cv+2, "test_Pr_matrix")

    for i in range(6):
        l+=1
        for j in range(cv):
            worksheet.write(l, j, _Pr_matrix[i,j])
            worksheet.write(l, j+cv+2, Pr_matrix[i,j])
        worksheet.write(l, j+1, _Pr_mean[i]) 
        worksheet.write(l, j+2, _Pr_SD[i])
        worksheet.write(l, j+2+cv+1, Pr_mean[i])
        worksheet.write(l, j+2+cv+2, Pr_SD[i])
    l+=1
    for j in range(cv):
        worksheet.write(l, j, _Pr_cv[j])
        worksheet.write(l, j+cv+2, Pr_cv[j])
    worksheet.write(l, j+1, np.mean(_Pr_cv), style=style)
    worksheet.write(l, j+2, np.std(_Pr_cv), style=style)
    worksheet.write(l, j+2+cv+1, np.mean(Pr_cv), style=style)
    worksheet.write(l, j+2+cv+2, np.std(Pr_cv), style=style)

    #=======#
    l+=3
    worksheet.write(l,0, "train_Rc_matrix")
    worksheet.write(l,cv+2, "test_Rc_matrix")

    for i in range(6):
        l+=1
        for j in range(cv):
            worksheet.write(l, j, _Rc_matrix[i,j])
            worksheet.write(l, j+cv+2, Rc_matrix[i,j])
        worksheet.write(l, j+1, _Rc_mean[i]) 
        worksheet.write(l, j+2, _Rc_SD[i])
        worksheet.write(l, j+2+cv+1, Rc_mean[i])
        worksheet.write(l, j+2+cv+2, Rc_SD[i])
    l+=1
    for j in range(cv):
        worksheet.write(l, j, _Rc_cv[j])
        worksheet.write(l, j+cv+2, Rc_cv[j])
    worksheet.write(l, j+1, np.mean(_Rc_cv), style=style)
    worksheet.write(l, j+2, np.std(_Rc_cv), style=style)
    worksheet.write(l, j+2+cv+1, np.mean(Rc_cv), style=style)
    worksheet.write(l, j+2+cv+2, np.std(Rc_cv), style=style)
    #=======#
    l+=3
    worksheet.write(l,0, "train_F1_matrix")
    worksheet.write(l,cv+2, "test_F1_matrix")

    for i in range(6):
        l+=1
        for j in range(cv):
            worksheet.write(l, j, _F1_matrix[i,j])
            worksheet.write(l, j+cv+2, F1_matrix[i,j])
        worksheet.write(l, j+1, _F1_mean[i]) 
        worksheet.write(l, j+2, _F1_SD[i])
        worksheet.write(l, j+2+cv+1, F1_mean[i])
        worksheet.write(l, j+2+cv+2, F1_SD[i])
    l+=1
    for j in range(cv):
        worksheet.write(l, j, _F1_weighted_cv[j])
        worksheet.write(l, j+cv+2, F1_weighted_cv[j])
    worksheet.write(l, j+1, np.mean(_F1_weighted_cv), style=style)
    worksheet.write(l, j+2, np.std(_F1_weighted_cv), style=style)
    worksheet.write(l, j+2+cv+1, np.mean(F1_weighted_cv), style=style)
    worksheet.write(l, j+2+cv+2, np.std(F1_weighted_cv), style=style)


    workbook.save("../res/DCF"+str(cols)+".xls")

[ 2019-07-06 00:09:57,950][cascade_classifier.fit_transform] X_groups_train.shape=[(26886, 2)],y_train.shape=(26886,),X_groups_test.shape=no_test,y_test.shape=no_test
[ 2019-07-06 00:09:57,952][cascade_classifier.fit_transform] group_dims=[2]
[ 2019-07-06 00:09:57,952][cascade_classifier.fit_transform] group_starts=[0]
[ 2019-07-06 00:09:57,954][cascade_classifier.fit_transform] group_ends=[2]
[ 2019-07-06 00:09:57,955][cascade_classifier.fit_transform] X_train.shape=(26886, 2),X_test.shape=(0, 2)
[ 2019-07-06 00:09:57,958][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(26886, 2), X_cur_test.shape=(0, 2)


DCF(0, 1)
GCForest


[ 2019-07-06 00:09:58,196][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_0.predict)=43.43%
[ 2019-07-06 00:09:58,418][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_1.predict)=43.55%
[ 2019-07-06 00:09:58,643][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_2.predict)=44.19%
[ 2019-07-06 00:09:58,866][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_3.predict)=43.24%
[ 2019-07-06 00:09:59,087][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_4.predict)=45.05%
[ 2019-07-06 00:09:59,094][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_cv.predict)=43.92%
[ 2019-07-06 00:09:59,336][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_1 - 5_folds.train_0.predict)=43.09%
[ 2019-07-06 00:09:59,566][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_1 - 5_folds

[ 2019-07-06 00:10:10,208][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_0.predict)=47.37%
[ 2019-07-06 00:10:10,776][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_1.predict)=48.09%
[ 2019-07-06 00:10:11,353][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_2.predict)=48.94%
[ 2019-07-06 00:10:11,917][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_3.predict)=47.42%
[ 2019-07-06 00:10:12,484][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_4.predict)=47.71%
[ 2019-07-06 00:10:12,491][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_cv.predict)=47.91%
[ 2019-07-06 00:10:13,100][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds.train_0.predict)=48.56%
[ 2019-07-06 00:10:13,963][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds

[ 2019-07-06 00:10:45,476][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_0.predict)=40.26%
[ 2019-07-06 00:10:45,921][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_1.predict)=38.87%
[ 2019-07-06 00:10:46,408][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_2.predict)=38.88%
[ 2019-07-06 00:10:47,150][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_3.predict)=40.52%
[ 2019-07-06 00:10:47,814][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_4.predict)=40.31%
[ 2019-07-06 00:10:47,822][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_cv.predict)=39.78%
[ 2019-07-06 00:10:48,308][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_0.predict)=38.99%
[ 2019-07-06 00:10:48,767][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds

...

[ 2019-07-06 01:03:38,323][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_0.predict)=87.49%
[ 2019-07-06 01:03:38,439][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_1.predict)=86.69%
[ 2019-07-06 01:03:38,557][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_2.predict)=86.09%
[ 2019-07-06 01:03:38,680][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_3.predict)=86.40%
[ 2019-07-06 01:03:38,804][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_4.predict)=86.09%
[ 2019-07-06 01:03:38,810][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_cv.predict)=86.55%
[ 2019-07-06 01:03:38,851][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_4 - 5_folds.train_0.predict)=85.15%
[ 2019-07-06 01:03:38,877][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_4 - 5_folds

[ 2019-07-06 01:03:54,306][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_0.predict)=82.53%
[ 2019-07-06 01:03:54,920][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_1.predict)=81.99%
[ 2019-07-06 01:03:55,461][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_2.predict)=82.21%
[ 2019-07-06 01:03:56,088][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_3.predict)=81.90%
[ 2019-07-06 01:03:56,758][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_4.predict)=82.28%
[ 2019-07-06 01:03:56,766][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_cv.predict)=82.18%
[ 2019-07-06 01:03:57,304][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_6 - 5_folds.train_0.predict)=82.84%
[ 2019-07-06 01:03:57,870][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_6 - 5_folds

[ 2019-07-06 01:04:22,192][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_7 - 5_folds.train_2.predict)=88.55%
[ 2019-07-06 01:04:23,175][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_7 - 5_folds.train_3.predict)=88.24%
[ 2019-07-06 01:04:24,122][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_7 - 5_folds.train_4.predict)=88.47%
[ 2019-07-06 01:04:24,129][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_7 - 5_folds.train_cv.predict)=88.47%
[ 2019-07-06 01:04:24,136][cascade_classifier.calc_f1] Weighted F1 (layer_2 - train.classifier_average)=88.51%
[ 2019-07-06 01:04:24,142][cascade_classifier.fit_transform] [layer=3] look_indexs=[0], X_cur_train.shape=(26892, 50), X_cur_test.shape=(0, 50)
[ 2019-07-06 01:04:24,527][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_0 - 5_folds.train_0.predict)=88.77%
[ 2019-07-06 01:04:24,899][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_0 - 5_folds.

[ 2019-07-06 01:04:45,810][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_1 - 5_folds.train_2.predict)=87.94%
[ 2019-07-06 01:04:46,204][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_1 - 5_folds.train_3.predict)=88.81%
[ 2019-07-06 01:04:46,589][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_1 - 5_folds.train_4.predict)=88.96%
[ 2019-07-06 01:04:46,595][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_1 - 5_folds.train_cv.predict)=88.40%
[ 2019-07-06 01:04:46,995][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds.train_0.predict)=87.95%
[ 2019-07-06 01:04:47,378][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds.train_1.predict)=89.34%
[ 2019-07-06 01:04:47,761][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds.train_2.predict)=87.67%
[ 2019-07-06 01:04:48,163][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds

[ 2019-07-06 01:05:09,561][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_3 - 5_folds.train_4.predict)=88.66%
[ 2019-07-06 01:05:09,568][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_3 - 5_folds.train_cv.predict)=88.45%
[ 2019-07-06 01:05:09,986][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_4 - 5_folds.train_0.predict)=80.49%
[ 2019-07-06 01:05:10,375][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_4 - 5_folds.train_1.predict)=79.78%
[ 2019-07-06 01:05:10,754][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_4 - 5_folds.train_2.predict)=80.29%
[ 2019-07-06 01:05:11,178][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_4 - 5_folds.train_3.predict)=80.72%
[ 2019-07-06 01:05:11,572][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_4 - 5_folds.train_4.predict)=80.10%
[ 2019-07-06 01:05:11,579][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_4 - 5_folds